# Solution #7 of Kaggle TrackML competition

by Yuval Reina, Trian Xylouris

in 2018


## Chapter 1: Methods and Results

The current paper demonstrates our solution to Kaggle's TrackML competition in 2018, which yielded a private leaderboard score of 0.8041 and place 7.

### Our basic method: A variant of the Hough transform

Due to the magnetic field in the experiment, the particles form helixes, which travel along the z-axis. Each hit in the x-y-z space can be the member of any particular helix in a certain family of helixes. The idea of the Hough transform is to go over each possible member of this family of helixes, and look at how many of all hits may belong to this particular member. The hits are then assigned to the helix (=track) with the most possible hits.

We employed this concept as follows. For each point (x,y,z) on a specific track with radius 1/(2*kt), which originates from (0,0,z0), the following two quantities will be roughly constant:

(1) Theta = arctan(y/x)+arcsin(kt*rr)

(2)	Gamma = (z-z0)*kt/dtheta

where rr=(x^2+y^2)^0.5. It remains to iterate over the radius and z0, store all Theta and Gamma for every hit in the x-y-z space, and put those hits together, whose Theta and Gamma are near each other.

To do the latter, we employ several techniques, which increase the speed and accuracy of our solution:
- use binning, i.e. divide the 3d space into many bins
- every 500 loops all hits belonging to tracks which are long enough are removed from the dataset; also the sizes of the bins are adjusted
- iterate over random pairs (kt,z0), where kt and z0 are taken from a normal 1d random distribution with means 7.5e-4 and 7.5
- adjust the size and location of each bin with a small random variation
- smooth out the values of (1) of (2) using sin/cos and arctan, respectively
- if two hits are on the same track and on the same sensor (= same combination of volume, layer, module), we remove the one hit of the two, whose parameters (1) and (2) are further away from the track's mean values of those parameters

The disadvantage of sparse binning over dbscan is it’s sensitivity, the advantages are its speed and its precision (almost no outliers). Our basic method, run once with 100.000 pairs, yields a score of ~0.73.

### Additional improvement 1: Machine Learning
We run the binning algorithm 8 times in total, and merge the resulting solutions, using a machine algorithm model. In particular, we use LightGBM, but expect that a deep neural network or even Logistic Regression would yield similar results. The machine learning algorithm predicts the probabilitiy that a track is good, based on what it has learned from the training events. In particular, it looks at 13 features for each track. This improvement increases the score from ~0.73 to ~0.76. 

Note: if we increased the number of merged solutions to 30, we expect to get ~0.78 instead of ~0.76.

### Additional improvement 2: Track Extension
We take the merged solution of improvement 1, and recalculate the (kt,z0) for every single track, such that the parameters (1) and (2) among the track members have minimal standard deviation. Using these refined (kt,z0) values, we extend every track, by assigning those hits onto it, such that parameters (1) and (2), if calculated with the refined (kt,z0), are very close to the track's mean values (or just close enough to at least one hit of the track). 

We run this operation also a second time, where we specifically look to re-assign hits, which have been assigned until then only to tracks of length 1 or 2.

This improvement increased the score from ~0.76 to 0.8041. If we hadn't used improvement 1, then this method improves the score from ~0.73 to ~0.79

### Outlook
Elements of our approach may be beneficial for any final solution to this problem. 

In particular, our binning algorithm creates a lot of good candidate tracks in a short period of time. It may be helpful to employ this at the beginning of a solution, in order to very quickly (<1 minute) detect 50% of all tracks (use small bins). One can then continue with running a different algorithm on the remaining hits.

Similarly, the machine learning algorithm, as well as the employed parameters and track extension algorithm may improve any final result, while just adding minutes (or less, after optimization) to the total runtime.

An obvious short-coming of our main approach is, that it is not suited to find tracks, which originate far from the origin. We did try to adjust our algorithm to work also in that situation, but had no success so far.

## Chapter 2: Running full pipeline for train event 1000

### Introduction

We will demonstrate how to get a score of TBD for train event 1000, in TBD minutes, by using our aforementioned method. The steps are: 
- Create 2 initial solutions: 2x use 5.500 pairs of (kt,z0) for binning (in our original solution, we use 100.000 pairs)
- Merge the 2 solutions using a machine learning algorithm (in our original solution, we use 8)
- Extend the tracks (in our original solution, we run this twice, while at the first time, we extend hits according to whether their values for (1) and (2) are close enough to at least 1 hit, instead of the mean of the track)

Import necessary packages and load train event 1000, which will be the event we will be working on:

In [61]:
from IPython.display import HTML
import numpy as np
import sys
sys.path.insert(0, 'other/')
import pandas as pd
import datetime
import os
from ipywidgets import FloatProgress,FloatText
from IPython.display import display
import time
import pdb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.cluster import DBSCAN
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from itertools import product
import gc
import cProfile
from tqdm import tqdm_notebook
%matplotlib inline
#make wider graphs
sns.set(rc={'figure.figsize':(12,5)})
plt.figure(figsize=(12,5))
path='files/'

from functions.other import calc_features, get_event, score_event_fast, load_obj
from functions.expand import *
from functions.cluster import *
from functions.ml_model import merge_with_probabilities,precision_and_recall,get_features,get_predictions

# the following two lines are for changing imported functions, and not needing to restart kernel to use their updated version
%load_ext autoreload
%autoreload 2

event_num=0
event_prefix = 'event00000100{}'.format(event_num)
hits, cells, particles, truth = get_event(path,event_prefix)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<Figure size 864x360 with 0 Axes>

### Clustering
Define parameters and run clustering, twice:

In [57]:
history=[]
weights={'pi':1,'theta':0.15}
stds={'z0':7.5, 'kt':7.5e-4}
d =    {'sint':[225,110,110,110,110,110],
        'cost':[225,110,110,110,110,110],
          'phi':[550,260,260,260,260,260],
        'min_group':[11,11,10,9,8,7],
        'npoints':[200,200,200,200,100,100]}  # quick run for testing
        #'npoints':[500,2000,1000,1000,500,500]}
filters=pd.DataFrame(d)
#nu=500
nu=100

resa1=clustering(hits,stds,filters,phik=3.3,nu=nu,truth=truth,history=history)
resa1["event_id"]=event_num
score = score_event_fast(truth, resa1.rename(index=str, columns={"label": "track_id"}))
print("Your score: ", score)

resa2=clustering(hits,stds,filters,phik=3.3,nu=nu,truth=truth,history=history)
resa2["event_id"]=event_num
score = score_event_fast(truth, resa2.rename(index=str, columns={"label": "track_id"}))
print("Your score: ", score)

FloatText(value=0.0, description='full score:')

FloatText(value=0.0, description='score:')

FloatText(value=0.0, description='s rate:')

FloatText(value=0.0, description='add score:')

FloatText(value=120939.0, description='Rest size:')

...

FloatText(value=120939.0, description='Rest size:')

FloatText(value=120939.0, description='Group size:')

FloatText(value=0.0, description='filter:')

100%|██████████| 1000/1000 [00:37<00:00, 26.32it/s]

took 38.06680 sec
Your score:  0.49943215791999995


## Employ Machine Learning
### General strategy
- Produce different submission candidates sub_1, sub_2, ..., sub_N
- Create a machine learning model, which gives probabilities between 0 and 1 for each track candidate
- Merge two submission candidates by assigning to each hit the track, which has higher probability
- Merge the submission candidates successively (sub_1 and sub_2 to sub_12, sub_12 and sub_3 to sub_123, etc.) to get the final submission

[Note: For our final solution, the methods described in this chapter gave around +0.01 to the LB score. Certain benefits from these methods have already been captured by the function, which expands the tracks.]

### Create machine learning model (LightGBM)
- Training data: Use the truth file of the train events 3-5 to get true tracks (target=1). To get wrong tracks (target=0) use generated clustering submissions on those latter events. In particular, we consider each track candidate, for which not all hits belong to the same particle_id, as a wrong track. Also, choose only tracks which have at least length 4, to slightly optimize compute time at negligible cost.
- Use 13 features per track: 
 - variance of x,y,z (these are the most important)
 - minimum of x,y,z
 - maximum of x,y,z
 - mean of z
 - volume_id of first hit 
 - number of clusters per track (i.e. are there many hits, which are close together)
 - number of hits / number of clusters   
- Validation data: Same as with training data, but use the 3 training events 0,1,2

[Note: We also tried many more features (compare below dataframe df_train), but they gave only small additional gains, so we just kept those 13 features for simplicity. In fact, many other interesting features, such as number of hits, number of different volumes crossed etc are closely related to the above used ones.]

We have prepared the training and test data and load it directly from a pkl-file:

In [46]:
df_train=load_obj('files/df_train_v2-reduced.pkl')
df_test=load_obj('files/df_test_v1.pkl')
y_train=df_train.target.values
y_test=df_test.target.values
print("The dataframe with all features:")
display(df_train.head())
print("Features for each track:",df_train.columns.values)

The dataframe with all features:


,cells_length,cells_max,cells_mean,cells_min,cells_var,elayer,emodule,evolume,ex,ey,...,ymean,ymedian,ymin,yvar,zmax,zmean,zmedian,zmin,zvar,event_id
0,4.0,0.341868,0.119788,2.044216e-02,0.017071,6.0,29.0,14.0,-290.533997,-463.497009,...,-256.535034,-286.803009,-463.497009,26005.789062,1801.500000,951.502136,1031.050049,107.703003,3.882222e+05,3
1,8.0,0.450497,0.071357,2.354972e-14,0.020859,2.0,145.0,12.0,-231.250000,351.147003,...,143.303238,92.862198,20.085199,10862.027344,-244.580994,-1334.507324,-958.000000,-2951.500000,7.111888e+05,3
2,11.0,0.680366,0.167157,1.636995e-02,0.035665,6.0,66.0,18.0,470.838989,768.648010,...,378.134766,309.616516,30.304199,72788.718750,1802.500000,782.384949,561.099976,64.440201,3.828446e+05,3
3,3.0,1.142238,0.461141,1.039750e-01,0.232131,12.0,26.0,18.0,94.234398,-865.289001,...,-450.889038,-434.042480,-865.289001,130411.406250,2947.500000,1532.954224,1469.185059,245.947006,1.497747e+06,3
4,10.0,0.791248,0.182531,3.750272e-02,0.052045,6.0,27.0,16.0,135.772003,-984.747986,...,-389.764557,-330.183014,-984.747986,91312.648438,-71.102097,-869.824463,-752.000000,-2152.500000,4.269275e+05,3


Features for each track: ['cells_length' 'cells_max' 'cells_mean' 'cells_min' 'cells_var' 'elayer'
 'emodule' 'evolume' 'ex' 'ey' 'ez' 'hitids' 'nclusters' 'nhits'
 'nhitspercluster' 'nsensors' 'nvolumes' 'particle_id' 'sentence'
 'sentence_count' 'sentence_value' 'slayer' 'smodule' 'svolume' 'sx' 'sy'
 'sz' 'target' 'xmax' 'xmean' 'xmedian' 'xmin' 'xvar' 'ymax' 'ymean'
 'ymedian' 'ymin' 'yvar' 'zmax' 'zmean' 'zmedian' 'zmin' 'zvar' 'event_id']


In the competition, we used roughly 250 events for training, but the additional improvement to just using 13 events is not too big. We now create the LightGBM model, using the mentioned training data and features:

In [47]:
import lightgbm
s=time.time()
# choose which features of the tracks we want to use:
columns=['svolume','nclusters', 'nhitspercluster', 'xmax','ymax','zmax', 'xmin','ymin','zmin', 'zmean', 'xvar','yvar','zvar']
rounds=1000
round_early_stop=50
parameters = { 'subsample_for_bin':800, 'max_bin': 512, 'num_threads':8, 
               'application': 'binary','objective': 'binary','metric': 'auc','boosting': 'gbdt',
               'num_leaves': 128,'feature_fraction': 0.7,'learning_rate': 0.05,'verbose': 0}
train_data = lightgbm.Dataset(df_train[columns].values, label=y_train)
test_data = lightgbm.Dataset(df_test[columns].values, label=y_test)
model = lightgbm.train(parameters,train_data,valid_sets=test_data,num_boost_round=rounds,early_stopping_rounds=round_early_stop,verbose_eval=50)
print('took',time.time()-s,'seconds')

Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.954964
[100]	valid_0's auc: 0.960705
[150]	valid_0's auc: 0.962809
[200]	valid_0's auc: 0.963605
[250]	valid_0's auc: 0.964108
[300]	valid_0's auc: 0.964196
[350]	valid_0's auc: 0.964214
Early stopping, best iteration is:
[317]	valid_0's auc: 0.964268
took 1.4954051971435547 seconds


### Judge machine learning model

We doublecheck the model's performance, by calculating its precision, recall and accuracy on the validation set:

[Note: For ML to be helpful in our situation, it needs to distinguish correct from wrong tracks with very high precision=true_positives/(false_positives+true_positives)). Also, it needs to do so for various sets of track candidates (especially such, which are generated if one tries to find tracks which originate far away from the origin; in those situations, often a lot of bad candidates are produced).]

In [50]:
y_test_pred=model.predict(df_test[columns].values)
precision, recall, accuracy=precision_and_recall(y_test, y_test_pred,threshold=0.1)
precision, recall, accuracy=precision_and_recall(y_test, y_test_pred,threshold=0.5)
precision, recall, accuracy=precision_and_recall(y_test, y_test_pred,threshold=0.9)

Threshold 0.1  --- Precision: 0.8458, Recall: 0.9955, Accuracy: 0.8861
Threshold 0.5  --- Precision: 0.9060, Recall: 0.9608, Accuracy: 0.9150
Threshold 0.9  --- Precision: 0.9621, Recall: 0.7437, Accuracy: 0.8251


### Use machine learning model

Calculate the probabilities for the tracks in those two submissions (small optimization: take also length of track into account, and after a couple of merged submissions, ask the probability of the track from the new submission to be at least C higher than the current probability; this latter option is not used in this kernel, but was used when merging >= 4 submissions). 

Then merge both submissions, based on the probabilities of its track candidates:

In [58]:
preds={}
preds[1]=get_predictions(resa1,hits,model)
preds[2]=get_predictions(resa2,hits,model)
print('Merge submission 0 and 1 into sub01:')
sub01=merge_with_probabilities(resa1,resa2,preds[1],preds[2],None,length_factor=0.5)
score = score_event_fast(truth, sub01)
print('Score:',score)

Merge submission 0 and 1 into sub01:
Score: 0.5651194312399999


## Expand tracks

In [60]:
mstd_vol={7:0,8:0,9:0,12:2,13:1,14:2,16:3,17:2,18:3}
mstd_size=[4,4,4,4,3,3,3,2,2,2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
weights={'theta':0.1, 'phi':1}
nresa=expand_tracks(sub01,hits,5,16,5,7,mstd=8,dstd=0.00085,phik=3.3,max_dtheta=0.9*np.pi/2,mstd_vol=mstd_vol,mstd_size=mstd_size,weights=weights,nhipo=100)
nresa['event_id']=0
score = score_event_fast(truth, nresa)
print("Your score: ", score)

100%|██████████| 100/100 [00:07<00:00, 14.28it/s]


FloatProgress(value=0.0, description='calculating:', max=6763.0)

100%|██████████| 6763/6763 [02:29<00:00, 45.12it/s]


Your score:  0.6942199117400001
